<a href="https://colab.research.google.com/github/MarinaChau/DataViz-Project-2022/blob/main/Projet_Data_Viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the environmental impact of my plate?

## Questions subsidiaires

* Quelle est la part de CO2 de la viande dans mon assiette?
* Faut-il choisir un produit conventionnel ou biologique?

**Agribalyse utilise le DQR score pour noter les produits:**

Une note de qualité - le Data Quality Ratio (DQR) - de 1, très bon, à 5, très mauvais - est associée à chaque produit agricole et alimentaire pour lequel Agribalyse fournit des inventaires de cycle de vie et des indicateurs d’impacts. La Commission Européenne recommande de la prudence dans l’utilisation des données avec des DQR supérieurs à 3. Dans la base de données AGRIBALYSE, 67 % des données ont un DQR jugé bon ou très bon (1 à 3).

Link to visualization library : `https://www.intellspot.com/python-visualization-tools/`

In [34]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, plot
from plotly.subplots import make_subplots
## fixer le theme
import plotly.io as pio
pio.templates.default = 'ggplot2'

# scikit-learn
## Métriques
from sklearn.metrics import silhouette_score
## Clustering
from sklearn.cluster import KMeans

# yellowbrick
## Clustering
from yellowbrick.cluster import SilhouetteVisualizer

In [25]:
# Download the different dataset

!wget https://www.data.gouv.fr/fr/datasets/r/83289c89-8134-4426-8959-eccad79f8873 # Summary of the dataset
!wget https://www.data.gouv.fr/fr/datasets/r/e822439e-e088-4f9f-b4d9-6fe10124a376 # Label description

!wget https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/metadata-attachments/AGRIBALYSE3_partie%20agriculture_conv.xlsx # raw data from industrial agriculture
!wget https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/metadata-attachments/AGRIBALYSE3_partie%20agriculture_bio.xlsx # raw data from organic agriculture

--2022-06-21 12:01:44--  https://www.data.gouv.fr/fr/datasets/r/83289c89-8134-4426-8959-eccad79f8873
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/raw [following]
--2022-06-21 12:01:44--  https://koumoul.com/data-fair/api/v1/datasets/agribalyse-synthese/raw
Resolving koumoul.com (koumoul.com)... 51.75.192.4
Connecting to koumoul.com (koumoul.com)|51.75.192.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 927255 (906K) [text/csv]
Saving to: ‘83289c89-8134-4426-8959-eccad79f8873.3’

83289c89-8134-4426- 100%[===================>] 905.52K  1.33MB/s    in 0.7s    

2022-06-21 12:01:45 (1.33 MB/s) - ‘83289c89-8134-4426-8959-eccad79f8873.3’ saved [927255/927255]

--2022-06-21 12:01:46--  https://www.data.gouv.fr/fr/datasets/r/e822439e-e088-4f9f-b

In [ ]:
df_synthese = pd.read_csv("/content/dataset_synth.csv")

## Exploratory Data Analysis

In [ ]:
df_synthese.head()

,Code AGB,Code CIQUAL,Groupe d'aliment,Sous-groupe d'aliment,Nom du Produit en Français,LCI Name,Saisonnalité,Transport par avion (1 : par avion),Livraison,Matériau d'emballage,...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
0,13712,13712,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, égoutté","Apricot, canned in light syrup, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.096413,0.009301,0.031766,0.410555,1.502775,44.823181,62.195218,1.630637,47.237697,2.938064
1,13713,13713,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, non égoutté","Apricot, canned in light syrup, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.059314,0.005726,0.019499,0.235366,0.797791,27.901738,36.285914,1.010960,29.397697,1.823726
2,13714,13714,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, égoutté","Apricot, in syrup, canned, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.096413,0.009301,0.031766,0.410555,1.502775,44.823181,62.195218,1.630637,47.237697,2.938064
3,13715,13715,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, non égoutté","Apricot, in syrup, canned, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.059314,0.005726,0.019499,0.235366,0.797791,27.901738,36.285914,1.010960,29.397697,1.823726
4,13000,13000,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru","Apricot, pitted, raw",mix de consommation FR,0,Ambiant (moyenne),LPDE,...,0.061462,0.007254,0.025954,0.204244,0.943669,43.838644,41.957928,1.412878,17.123820,2.173328


In [29]:
df_synthese.isna().sum() # No null

Code AGB                                                                    0
Code CIQUAL                                                                 0
Groupe d'aliment                                                            0
Sous-groupe d'aliment                                                       0
Nom du Produit en Français                                                  0
LCI Name                                                                    0
Saisonnalité                                                                0
Transport par avion (1 : par avion)                                         0
Livraison                                                                   0
Matériau d'emballage                                                        0
Préparation                                                                 0
DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)           0
Score unique EF (mPt/kg de produit)                             

In [52]:
df = df_synthese.drop_duplicates()

In [53]:
# We're going to drop "Code AGB" and "Code CIQUAL"
df = df.drop(['Code AGB', 'Code CIQUAL', 'LCI Name'], axis=1)

In [31]:
df.shape

(2479, 27)

In [32]:
df["Groupe d'aliment"].value_counts()

viandes, œufs, poissons                        636
fruits, légumes, légumineuses et oléagineux    423
produits céréaliers                            375
entrées et plats composés                      281
lait et produits laitiers                      220
boissons                                       219
aides culinaires et ingrédients divers         158
matières grasses                                56
produits sucrés                                 53
aliments infantiles                             33
glaces et sorbets                               25
Name: Groupe d'aliment, dtype: int64

In [33]:
df.columns

Index(['Code AGB', 'Code CIQUAL', 'Groupe d'aliment', 'Sous-groupe d'aliment',
       'Nom du Produit en Français', 'LCI Name', 'Saisonnalité',
       'Transport par avion (1 : par avion)', 'Livraison',
       'Matériau d'emballage', 'Préparation',
       'DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)',
       'Score unique EF (mPt/kg de produit)',
       'Changement climatique (kg CO2 eq/kg de produit)',
       'Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit)',
       'Rayonnements ionisants (kBq U-235 eq/kg de produit)',
       'Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit)',
       'Particules (E-06 disease inc./kg de produit)',
       'Acidification terrestre et eaux douces (mol H+ eq/kg de produit)',
       'Eutrophisation terreste (mol N eq/kg de produit)',
       'Eutrophisation eaux douces (E-03 kg P eq/kg de produit)',
       'Eutrophisation marine (E-03 kg N eq/kg de produit)',
       'Utilisation du sol (Pt/kg de p

In [44]:
df.shape  # 27 features and 2479 products

(2479, 25)

### Niveau d'exploration 2

**Analyse univariée**

In [36]:
target = "DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)"

In [45]:
# avec un nombre de bins = 4
fig = px.histogram(df, x=target, nbins=4,
                   histnorm='probability',
                   marginal='box',
                   title='Distribution du DQR',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
fig

In [46]:
# avec un nombre de bins élevé
fig = px.histogram(df, x=target, nbins=200,
                   histnorm='probability',
                   marginal='box',
                   title='Distribution du DQR',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
fig

* median = 2.76 : la moitié des DQR sont inférieures ou égale à 2.76.
* Les notes DQR inférieures strictement à 3 sont considérées comme bons ou très bons : cela consitituent environ de 67% des valeurs.
* NB : pour la valeur 3, ici elle est apparemment considérée comme fiable, et ici ce n'est pas le cas. J'ai considéré ici 3 comme peu fiable. Seule 22 produits ont un DQR égal à 3.
* Ils semblent qu'ils y aient des groupes de DQR (voir "5e niveau d'exploration").

In [47]:
# nombre de produits ayant un DQR=3
len(df[df_synthese.loc[:,target]==3])

22

In [48]:
# nombre de sous-groupes
df.loc[:,"Sous-groupe d'aliment"].nunique()

58

In [50]:
fig = px.histogram(df, y="Sous-groupe d'aliment", 
                  title="Répartition des différents sous-groupes d'aliment", orientation ="h",
                  color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_yaxes(categoryorder="total ascending")
fig


* Le nombre de catégories est plus important : il y a 58 sous-groupes.
* A cette échelle, avec ce nombre de catégories :
** chaque catégorie est assez peu représentée : le 1e sous-groupe ne représente même pas 10% des produits
plus de la moitié des sous-catégories représentent moins de 1% des produits.
** Regardons les sous-groupes qui contiennent au moins 100 produits (le diagramme ci-dessous précise le lien entre "Groupe d'aliment" et "Sous-groupe d'aliment") :
* Le premier sous-groupe est "légumes", qui appartient au 2e groupe "fruits, légumes, légumineuses et oléagineux", et non pas au 1e groupe
* Les 2e et 3e sous-groupes sont "charcuteries" et "viandes crues", qui appartiennent au 1e groupe "viandes, œufs, poissons".
* 4e sous-groupe : "céréales de petit-déjeuner et biscuits"
* Moins attendus, les sous-groupes "plats composés" et "fromages" appartiennent aux 4e groupe "entrées et plats composés" et au 5e groupe "lait et produits laitiers".
* Le 7e sous-groupe est "viandes cuites" du 1e groupe.

In [54]:
fig = px.icicle(df,
                path=[px.Constant("Tous les produits"), "Groupe d'aliment", "Sous-groupe d'aliment"], 
                title="Relation entre 'Groupe d'aliment' et 'Sous-groupe d'aliment'",
                color_discrete_sequence=px.colors.qualitative.Vivid
             )
fig.update_traces(root_color='lightgrey')
fig

### Livraison

In [57]:
fig = px.histogram(df, x="Livraison", 
                  title="Répartition des différents types de livraison", 
                  histnorm='probability',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_xaxes(categoryorder="total descending")
fig

In [61]:
fig = px.icicle(df.loc[df.Livraison=='Glacé'],
                path=['Livraison', "Groupe d'aliment", "Sous-groupe d'aliment"],
                color_discrete_sequence=px.colors.qualitative.Vivid, 
                title="Livraison 'Glacé : groupes et sous-groupes d'aliments concernés"
               )
fig.update_traces(root_color='lightgrey')

In [62]:
fig = px.icicle(df.loc[df.Livraison=='Ambiant (moyenne)'],
                path=['Livraison', "Groupe d'aliment", "Sous-groupe d'aliment"],
                color_discrete_sequence=px.colors.qualitative.Vivid,
                title="Livraison 'Ambiant (moyenne)' : groupes et sous-groupes d'aliments concernés"
               )
fig.update_traces(root_color='lightgrey')

In [63]:
fig = px.icicle(df.loc[df.Livraison=='Ambiant (long)'],
          path=['Livraison', "Groupe d'aliment", "Sous-groupe d'aliment"],
               color_discrete_sequence=px.colors.qualitative.Vivid,
                title="Livraison 'Ambiant (long)' : groupes et sous-groupes d'aliments concernés"
               )
fig.update_traces(root_color='lightgrey')

* Près de la moitié des livraisons 'Glacé' sont du groupe d'aliment 'viandes, oeufs, poissons', qui joue encore un rôle prépondérant.
* Pour les livraison "Ambiant (moyenne)", "fruits, légumes, légumineuses et oléagineux" et "boissons" sont les groupes prépondérants.
* Pour les livraison "Ambiant (long)", "produits céréaliers" est le groupe prépondérant.
*Notons la relation d'ordre entre 'Livraison' et 'Groupe d'aliment' :
 * 'Glacé' / 'viandes, oeufs, poissons'
 * "Ambiant (moyenne)" / "fruits, légumes, légumineuses et oléagineux"
 * "Ambiant (long)" / "produits céréaliers".

### Matériau d'emballage

In [65]:
fig = px.histogram(df, x="Matériau d'emballage", 
                    title="Répartition des différents materiau d'emballage", 
                  #  histnorm='probability',
                    color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_xaxes(categoryorder="total descending")
fig

Signification des Sigles
(source : internet)

* PS = Polystyrène (plastique)
* LDPE = polyéthylène à basse densité (plastique)
* PP = polypropylène (plastique)
* PET = PETE = polyéthylène téréphtalate (plastique)
* PVC = chlorure de polyvinyle (plastique)
* HPDE = High-density polyethylene (plastique)
* PE = polyéthylène (plastique)

In [66]:
plastique_emballage = ['PS', 'LPDE', 'PP', 'Déjà emballé - PET', 'V (PVC)', 'Bouteille PETE',  'HPDE',
                       'Déjà emballé - PP/PE', 'Déjà emballé - LDPE', 'PVC', 'Bouteille PET',]

In [68]:
table = df.loc[:,["Matériau d'emballage"]].isin(plastique_emballage)
len(table[table["Matériau d'emballage"]==True])/len(df)

0.7067365873336022

70% des produits présents dans la base sont emballés avec du plastique.

In [69]:
# on conserve que les produits contenant ces emballages
df_plastique = df[(df.loc[:,["Matériau d'emballage"]].isin(plastique_emballage)).squeeze()]


fig = px.icicle(df_plastique,
                path=[px.Constant("Matériau d'emballage plastique"), "Groupe d'aliment", "Sous-groupe d'aliment"],
                color_discrete_sequence=px.colors.qualitative.Vivid,
                title="Matériau d'emballage plastique : groupes et sous-groupes d'aliments concernés"
               )
fig.update_traces(root_color='lightgrey')

* Le groupe d'aliment "viandes, oeufs, poissons" sont les plus gros consommateurs d'emballage plastique, près du quart des plastiques.
* Viennent ensuite les groupes d'aliment :
 * "produits céréaliers" (> 10%)
 * "lait et produits laitiers" (> 10%)
 * " fruits, légumes, légumineuses et oléagineux" (> 10%).
* Comme "viandes, oeufs, poissons" est le groupe d'aliment majoritaire, on peut s'attendre à une représentation importante de ce groupe parmi les produits utilsant des emballages plastiques.
* Dans ce groupe "viandes, oeufs, poissons", quelle est la proportion d'emballages plastiques utilisés ?

In [72]:
# on conserve que le groupe 'viandes, œufs, poissons'
df_gp1_viandes = df[df.loc[:,"Groupe d'aliment"]=='viandes, œufs, poissons']
# proportion d'emballages plastiques utilisées dans ce groupe
len(df_gp1_viandes[df_gp1_viandes.loc[:,"Matériau d'emballage"].isin(plastique_emballage)])/len(df_gp1_viandes)

0.9072327044025157

In [76]:
# liste des colonnes numériques, sans la target
col_float_no_target = df.drop('DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)', axis=1).select_dtypes(float).columns
col_float_no_target

Index(['Score unique EF (mPt/kg de produit)',
       'Changement climatique (kg CO2 eq/kg de produit)',
       'Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit)',
       'Rayonnements ionisants (kBq U-235 eq/kg de produit)',
       'Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit)',
       'Particules (E-06 disease inc./kg de produit)',
       'Acidification terrestre et eaux douces (mol H+ eq/kg de produit)',
       'Eutrophisation terreste (mol N eq/kg de produit)',
       'Eutrophisation eaux douces (E-03 kg P eq/kg de produit)',
       'Eutrophisation marine (E-03 kg N eq/kg de produit)',
       'Utilisation du sol (Pt/kg de produit)',
       'Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit)',
       'Épuisement des ressources eau (m3 depriv./kg de produit)',
       'Épuisement des ressources énergétiques (MJ/kg de produit)',
       'Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)'],
      dtype='object')

In [78]:
# coefficient d'asymétrie

skew_list=[]
for col in col_float_no_target:
    skew_list.append((df[col].std()))
    
pd.DataFrame(skew_list, index=col_float_no_target)

,0
Score unique EF (mPt/kg de produit),0.885750
Changement climatique (kg CO2 eq/kg de produit),9.138402
Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),4.913065
Rayonnements ionisants (kBq U-235 eq/kg de produit),1.898260
Formation photochimique d'ozone (E-03 kg NMVOC eq/kg de produit),42.971933
Particules (E-06 disease inc./kg de produit),0.886995
Acidification terrestre et eaux douces (mol H+ eq/kg de produit),0.131431
Eutrophisation terreste (mol N eq/kg de produit),0.552303
Eutrophisation eaux douces (E-03 kg P eq/kg de produit),2.567846
Eutrophisation marine (E-03 kg N eq/kg de produit),50.225766


## Analyse selon deux variables

In [80]:
# liste des variables type 'object'  
object_col = df.select_dtypes(object).columns

# afficher les box-plot
for col in object_col:
    fig = px.box(df, x=col, y=target, color=col, 
                 title="DQR en fonction de '{}'".format(col),
                 color_discrete_sequence=px.colors.qualitative.Vivid
                )
    fig.add_hline(y=3, line_width=1, line_color="red")
    fig.update_layout(boxgap=0, showlegend=False)
    fig.update_xaxes(tickangle=45)
    fig.update_traces(boxmean=True)
    iplot(fig)